In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
from sklearn import tree
from IPython.display import Image
import pydotplus as pydot 
warnings.filterwarnings("ignore")
from sklearn.externals.six import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from numpy import set_printoptions
import tensorflow as tf
set_printoptions(precision=3)

In [6]:
Data = pd.read_excel("Gen.xlsx",sheet_name="data").dropna(how = "any")
Data['Generator 1capacity (KVA)'] = Data['Generator 1capacity (KVA)'].astype("category")
Data = Data.dropna(how = 'any')

In [13]:
Data['consumption_perDay_within_a_period'] = Data['Consumption HIS']/(Data['NBRE DE JOUR'])
Data.loc[~np.isfinite(Data['consumption_perDay_within_a_period']), 'consumption_perDay_within_a_period'] = 0
Data['Quanitity_consumed_btn_visits'] = Data['QTE FUEL PRECEDENTE'] - Data['QTE FUEL TROUVEE']
set_printoptions(precision=3)

In [ ]:
Data.drop(['Power Type'],axis=1, inplace=True)
Data.drop(['ANCIEN COMPTEUR HORAIRE G1'],axis=1, inplace=True)
Data.drop(['Class'],axis=1, inplace=True)
Data.drop(['DATE  EFFECTIVE DE LA VISITE'],axis=1, inplace=True)
Data.drop(['DATE DE VISITE PRECEDENTE'],axis=1, inplace=True)

Data.drop(['Months'],axis=1, inplace=True)
Data.drop(['Cluster'],axis=1, inplace=True)
Data.drop(['Site Name'],axis=1, inplace=True)
Data.drop(['Generator 1capacity (KVA)'],axis=1, inplace=True)

Data.drop(['COMPTEUR HORAIRE ACTUEL GE1'],axis=1, inplace=True)
Data.drop(['QTE FUEL AJOUTEE'],axis=1, inplace=True)


In [15]:

Data['Running time per day'] = Data["NBRE HEURES DE FONCTIONNEMENT DG1"]/Data["NBRE DE JOUR"]             
Data.loc[~np.isfinite(Data['Running time per day']), 'Running time per day'] = 0
set_printoptions(precision=3)

In [16]:
Data['Maximum_consumption_perDay'] = Data[['CONSOMMATION HORAIRE DE FUEL DG1']]*24
set_printoptions(precision=3)

In [17]:
a = list(Data['NBRE HEURES DE FONCTIONNEMENT DG1'])
b = list(Data['Quanitity_consumed_btn_visits'])

In [18]:
c = []
for i in range(len(a)):
    if a[i] == 0  and  b[i]!=0:
        c.append(0)
    else:
        c.append(1)

In [19]:
t = list(Data['Maximum_consumption_perDay'])
r = list (Data['consumption_perDay_within_a_period'])

In [20]:
z = []
for i in range(len(r)):
    if r[i] > t[i]:
        z.append(0)
    else:
        z.append(1)

In [21]:
p = []
for i in range(len(z)):
    if c[i]==1 and z[i]==1:
        p.append(1)
    else:
        p.append(0)     

In [22]:
Data['Class'] = p


In [25]:
Data.drop(['QTE TOTALE LAISSEE'],axis=1, inplace=True)
#Data.drop(['Consumption HIS'],axis=1, inplace=True)

Data.drop(['QTE FUEL PRECEDENTE'],axis=1, inplace=True)
Data.drop(['QTE FUEL TROUVEE'],axis=1, inplace=True)
#Data.drop(['NBRE HEURES DE FONCTIONNEMENT DG1'],axis=1, inplace=True)
#Data.drop(['CONSOMMATION HORAIRE DE FUEL DG1'],axis=1, inplace=True)
Data.drop(['NBRE DE JOUR'],axis=1, inplace=True)
Data.drop(['Quanitity_consumed_btn_visits'],axis=1, inplace=True)
Data.drop(['Maximum_consumption_perDay'],axis=1, inplace=True)

In [ ]:
With this number of varibales consider to determin thr classification prediction.

In [26]:
Data.columns

Index(['Consumption HIS', 'NBRE HEURES DE FONCTIONNEMENT DG1',
       'CONSOMMATION HORAIRE DE FUEL DG1',
       'consumption_perDay_within_a_period', 'Running time per day', 'Class'],
      dtype='object')

In [27]:

# FUNCTION TO MAKE PREDICTIONS

def predictmodel(model):
    y_pred = model.predict(x_test)
    f,t,thresholds = metrics.roc_curve(y_test,y_pred)
    cm = metrics.confusion_matrix(y_test,y_pred)
    
    print("Score:", metrics.auc(f,t))
    print("Classification report:")
    print(metrics.classification_report(y_test,y_pred))
    print("Confusion Matrix:")
    print(cm)

With the following i use relu activation function and defult solver and a learning rate adaptives.

In [32]:
x = Data.loc[:,Data.columns.tolist()[0:5]]
x = x.as_matrix()
Y = Data.loc[:,'Class']
Y = Y.as_matrix()
x_train,x_test,y_train,y_test = train_test_split(x,Y,test_size=0.33,random_state=2700)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test) 

clf = MLPClassifier( activation='relu', beta_1=0.6, beta_2=0.9,alpha = 0.001,early_stopping = True,shuffle = True,warm_start = True,validation_fraction = 0.3,learning_rate_init=0.01, max_iter = 14000, random_state = 2700, learning_rate='adaptive')
clf.fit(x_train , y_train)


predictmodel(clf)
print(x.shape)
print(Y.shape)
print(x_test.shape)
print( x_train.shape)
print( y_test.shape)
print(y_train.shape)

Score: 0.8578711300732962
Classification report:
             precision    recall  f1-score   support

          0       0.85      0.80      0.82       330
          1       0.88      0.92      0.90       554

avg / total       0.87      0.87      0.87       884

Confusion Matrix:
[[263  67]
 [ 45 509]]
(2676, 5)
(2676,)
(884, 5)
(1792, 5)
(884,)
(1792,)


In [34]:
y_pred= clf.predict(x_test)
print('Overall AUC:', roc_auc_score(y_test, clf.predict(x_test)))

submission = pd.DataFrame({"Original":y_test,"predicted":y_pred})

Overall AUC: 0.8578711300732962


In [35]:
submission.head()

,Original,predicted
0,1,1
1,1,0
2,1,1
3,1,1
4,1,1
